In [0]:
from pyspark.sql import functions as F

bronze = spark.table("project.taxi_bronze.trips_raw")

base = (
    bronze
    .withColumn("pickup_ts", F.to_timestamp("tpep_pickup_datetime"))
    .withColumn("dropoff_ts", F.to_timestamp("tpep_dropoff_datetime"))
    .withColumn("pickup_date", F.to_date("pickup_ts"))
    .withColumn("pickup_year", F.year("pickup_ts"))
    .withColumn("pickup_month", F.month("pickup_ts"))
    .withColumn(
        "trip_minutes",
        (F.unix_timestamp("dropoff_ts") - F.unix_timestamp("pickup_ts")) / 60.0
    )
)


In [0]:
quarantine = (
    base
    .withColumn(
        "dq_reason",
        F.when(F.col("pickup_ts").isNull() | F.col("dropoff_ts").isNull(), F.lit("NULL_TIMESTAMP"))
         .when(F.col("trip_minutes") <= 0, F.lit("NEGATIVE_OR_ZERO_DURATION"))
         .when(F.col("trip_minutes") > 240, F.lit("DURATION_TOO_LONG"))
         .when(F.col("trip_distance") <= 0, F.lit("NONPOSITIVE_DISTANCE"))
         .when(F.col("total_amount") <= 0, F.lit("NONPOSITIVE_TOTAL_AMOUNT"))
         .when(
             (F.col("pickup_year") != F.col("year")) | (F.col("pickup_month") != F.col("month")),
             F.lit("EVENT_TIME_OUTSIDE_SOURCE_PARTITION")
         )
    )
    .filter(F.col("dq_reason").isNotNull())
    .withColumn("dq_detected_ts", F.current_timestamp())
    .select(
        "pickup_ts",
        "dropoff_ts",
        "pickup_date",
        "pickup_year",
        "pickup_month",
        "year",
        "month",
        "dq_reason",
        "dq_detected_ts",
        F.col("PULocationID").cast("int").alias("pickup_location_id"),
        F.col("DOLocationID").cast("int").alias("dropoff_location_id"),
        F.col("trip_distance").cast("double"),
        F.col("fare_amount").cast("double"),
        F.col("tip_amount").cast("double"),
        F.col("total_amount").cast("double"),
        "trip_minutes"
    )
)


In [0]:
spark.sql("DROP TABLE IF EXISTS project.taxi_silver.trips_quarantine")

(quarantine.write
  .format("delta")
  .mode("overwrite")
  .partitionBy("dq_reason")
  .saveAsTable("project.taxi_silver.trips_quarantine")
)


In [0]:
%sql
SELECT dq_reason, COUNT(*) AS rows
FROM project.taxi_silver.trips_quarantine
GROUP BY dq_reason
ORDER BY rows DESC;


dq_reason,rows
NONPOSITIVE_DISTANCE,59738
NONPOSITIVE_TOTAL_AMOUNT,25017
DURATION_TOO_LONG,6226
NEGATIVE_OR_ZERO_DURATION,5225
EVENT_TIME_OUTSIDE_SOURCE_PARTITION,96
